In [1]:
#!/usr/bin/env python
# encoding: utf-8
from __future__ import absolute_import, division, print_function, unicode_literals
# Install TensorFlow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator



gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)



from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
# from xgboost import XGBClassifier
import tensorflow.keras.backend as K
from sklearn import metrics

# !pip3 install keras-tuner --upgrade
# !pip3 install autokeras
import kerastuner as kt
import autokeras as ak

# Import local libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import importlib
import os
from tqdm import tqdm 

import logging

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

os.environ['NUMEXPR_MAX_THREADS'] = '64'
os.environ['NUMEXPR_NUM_THREADS'] = '64'

print("Tensorflow Version is {}".format(tf.__version__))
print("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')
!nvidia-smi

1 Physical GPUs, 1 Logical GPU
Tensorflow Version is 2.4.1
Keras Version is 2.4.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11527863447805920054
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10485760000
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 5247363364557777493
physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:c1:00.0, compute capability: 8.6"
]
Mon Sep 13 01:13:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|=========================

In [2]:
%%time
HOMEPATH = "/dicos_ui_home/alanchung/Universality_Boosetd_Higgs/"

data_train = {
#             "herwig_ang_train" : 0,
#             "pythia_def_train" : 0,
#             "pythia_vin_train" : 0,
        "pythia_dip_train" : 0,
#             "sherpa_def_train" : 0
        }  

for i, element in enumerate(data_train):
#         data_train[element] = pd.read_csv(savepath + "BDT/" + str(element) + ".csv")

    """
    Pt Range Study
    """
    pt_min, pt_max = 300, 1100
    tmp = pd.read_csv(HOMEPATH + "Notebook/KFold/" + str(element) + ".csv")
    tmp = tmp[(tmp["PTJ_0"] >= pt_min)  & (tmp["PTJ_0"] < pt_max)]
    tmp = tmp[(tmp["MJ_0"] >= 110)  & (tmp["MJ_0"] < 160)]
    data_train[element] = shuffle(tmp)



logging.info("All Files are loaded!")

logging.info("H jet : QCD jet = 1 : 1")
logging.info("\r")


train = [ len(data_train[element]) for j, element in enumerate(data_train)]
logging.info("{:^8}{:^15}".format("",str(element)))
logging.info("{:^8}{:^15}".format("Train #",train[0]))


for i, element in enumerate(data_train):
    total_list = data_train[element].columns
    break

logging.info("total_list: {}".format(total_list))





features = ["MJ_0","t21_0","D21_0","D22_0","C21_0","C22_0"] #7/14

DNN_Model_A1 = {
#               "herwig_ang" : 0,
#               "pythia_def" : 0, 
#               "pythia_vin" : 0, 
              "pythia_dip" : 0, 
#               "sherpa_def" : 0,
            }


kf = KFold(n_splits = 10)

for i,(model, trainingdata) in enumerate(zip(DNN_Model_A1, data_train)):

    logging.info("DNN Model: {}  Training Data: {}".format(model, trainingdata))
    
    for model_index, (train_index, val_index) in enumerate(kf.split(data_train[trainingdata]["target"])):
        training_data = data_train[trainingdata].iloc[train_index]
        validation_data = data_train[trainingdata].iloc[val_index]

        filepath = "./Pythia_dip_KFold/DNN_"+str(model)+"_Models_"+str(int(pt_min))+str(int(pt_max))+"/" + str(model) + "_DNN_"+str(model_index)+ ".h5"
        if os.path.exists(filepath):
            DNN_Model_A1[model] = load_model(filepath)
            logging.info(str(model) + " DNN model 1 is loaded!")
            logging.info("######################################################################################")
            logging.info("\n")
            
            
        else:
            raise ValueError("Pleas check model path !!")

INFO:root:All Files are loaded!
INFO:root:H jet : QCD jet = 1 : 1
INFO:root:
INFO:root:        pythia_dip_train
INFO:root:Train #     268760     
INFO:root:total_list: Index(['GEN', 'SHO', 'PRO', 'MJ_0', 'PTJ_0', 'eta_0', 'phi_0', 't21_0',
       'D21_0', 'D22_0', 'C21_0', 'C22_0', 'MJ', 'PTJ', 'eta', 'phi', 't21',
       'D21', 'D22', 'C21', 'C22', 'weight', 'eventindex', 'WEIGHT', 'index',
       'target'],
      dtype='object')
INFO:root:DNN Model: pythia_dip  Training Data: pythia_dip_train


ValueError: Pleas check model path !!

In [ ]:
%%time
HOMEPATH = "/dicos_ui_home/alanchung/Universality_Boosetd_Higgs/"
Data_High_Level_Features_path =  HOMEPATH + "Data_High_Level_Features/"
savepath = HOMEPATH + "Data_ML/"



data_train = {
        "herwig_ang_train" : 0,
        "pythia_def_train" : 0,
        "pythia_vin_train" : 0,
        "pythia_dip_train" : 0,
#             "sherpa_def_train" : 0
        }  

data_test = {
        "herwig_ang_test" : 0,
        "pythia_def_test" : 0,
        "pythia_vin_test" : 0,
        "pythia_dip_test" : 0,
#             "sherpa_def_test" : 0
        }  


DNN_Model_A1 = {
      "herwig_ang" : 0,
      "pythia_def" : 0, 
      "pythia_vin" : 0, 
      "pythia_dip" : 0, 
#       "sherpa_def" : 0,
    }

       
for i, model in enumerate(DNN_Model_A1): 
    logging.info("DNN Model: {}".format(model))

    for j, (traindata, testdata) in enumerate(zip(data_train,data_test)):


        """
        Pt Range Study
        """
    #         for k, (pt_min, pt_max) in  enumerate(zip([300,400,500,600,700,800],[400,500,600,700,800,900])):
        for k, (pt_min, pt_max) in  enumerate(zip([300],[500])):

            train_data_path = HOMEPATH + "Notebook/KFold/" + str(traindata) + ".csv"
            if os.path.exists(train_data_path):
                tmp = pd.read_csv(train_data_path)
            else:
                raise ValueError("Pleas check training data path !!")

            tmp = tmp[(tmp["PTJ_0"] >= pt_min)  & (tmp["PTJ_0"] < pt_max)]
#             tmp = tmp[(tmp["PTJ_0"] >= 300)  & (tmp["PTJ_0"] < 500)]
            tmp = tmp[(tmp["MJ_0"] >= 110)  & (tmp["MJ_0"] < 160)]
            data_train[traindata] = shuffle(tmp)#[:30000]


            test_data_path = HOMEPATH + "Notebook/KFold/" + str(testdata) + ".csv"
            if os.path.exists(test_data_path):
                tmp = pd.read_csv(test_data_path)
            else:
                raise ValueError("Pleas check test data path !!")

            tmp = tmp[(tmp["PTJ_0"] >= pt_min)  & (tmp["PTJ_0"] < pt_max)]
            tmp = tmp[(tmp["MJ_0"] >= 110)  & (tmp["MJ_0"] < 160)]
            data_test[testdata] = shuffle(tmp)

            logging.info("START===========================================START")
            logging.info("\r") 
            logging.info("All Files are loaded!")
            logging.info("pt min: {} , pt max: {}".format(pt_min, pt_max))
            logging.info("\r")
            logging.info("H jet : QCD jet = 1 : 1")

            train = len(data_train[traindata])
            test = len(data_test[testdata])
        #     logging.info("{:^8}{:^15}{:^15}{:^15}{:^15}".format("","Herwig Angular","Pythia Default","Pythia Vincia","Pythia Dipole"))
        #     logging.info("{:^8}{:^15}{:^15}{:^15}{:^15}".format("Train #",train[0],train[1],train[2],train[3]))
    #         logging.info("{:^8}{:^15}".format("",str(testdata)))
            logging.info("{:^8}{:^15}".format("Train #",train))
            logging.info("{:^8}{:^15}".format("Test #",test))






            features = ["MJ_0","t21_0","D21_0","D22_0","C21_0","C22_0"] 


            kf = KFold(n_splits = 100)
            # kf = GroupKFold(n_splits = 100)
#             skf = StratifiedKFold(n_splits = 2, random_state = 7, shuffle = True) 

#             model = "pythia_def"

            logging.info("DNN Model: {}  Test Data: {}".format(model, traindata))
            time.sleep(0.5)
            for model_index, (train_index, val_index) in tqdm(enumerate(kf.split(data_train[traindata]["target"]))):
                ticks_1 = time.time()

    #             logging.info("TRAIN: {} VAL: {}".format(train_index, val_index))
    #                 training_data = data_train[traindata].iloc[train_index]
                validation_data = data_train[traindata].iloc[val_index]
#                 logging.info(len(validation_data[validation_data["target"]==0]))
    #             validation_data = data_train[traindata]
    #                 validation_data = data_test[testdata]

        #         print(len(training_data[training_data["target"]==1])/len(training_data))
        #         print(len(validation_data[validation_data["target"]==0])/len(validation_data))
        #         print(len(training_data)/len(data_train[trainingdata]))
        #         print(len(validation_data)/len(data_train[trainingdata]))
        #         print(model_index)

                Performance_Frame = {
                "AUC" : [0],
                "max_sig" : [0],
                "r05" : [0],
                "time": [0]
                }


                filepath = "./"+str(model)+"_KFold/DNN_"+str(model)+"_Models_"+str(int(pt_min))+str(int(pt_max))+"/" + str(model) + "_DNN_"+str(model_index)+ ".h5"
                if os.path.exists(filepath):
    #                     DNN_Model_A1[model] = load_model(filepath)

                    model_DNN = load_model(filepath)

                    prediction_test =  model_DNN.predict(np.asarray(validation_data[features]))
                    discriminator_test = prediction_test
                    discriminator_test = discriminator_test/(max(discriminator_test))

                    Performance_Frame["AUC"][0] = metrics.roc_auc_score(validation_data["target"], discriminator_test)
                    FalsePositiveFull, TruePositiveFull, _ = metrics.roc_curve(validation_data["target"], discriminator_test)
                    tmp = np.where(FalsePositiveFull != 0)
                    Performance_Frame["max_sig"][0] = max(TruePositiveFull[tmp]/np.sqrt(FalsePositiveFull[tmp])) 
                    tmp = np.where(TruePositiveFull >= 0.5)
                    Performance_Frame["r05"][0]= 1./FalsePositiveFull[tmp[0][0]]

                    Performance_Frame["time"][0] = (time.time() - ticks_1)/60.

                    dataframe = pd.DataFrame(Performance_Frame)



                    csvdata_path = "./"+str(model)+"_KFold/DNN_"+str(model)+"_Performance_Table_to_"+str(traindata)+"_"+str(pt_min)+str(pt_max)+".csv"
                    if os.path.exists(csvdata_path):
                        save_to_csvdata = pd.read_csv(csvdata_path)
                        DATA = pd.concat([save_to_csvdata, dataframe], ignore_index=True, axis=0,join='inner')
                        DATA.to_csv(csvdata_path, index = 0)

                    else:
                        dataframe.to_csv(csvdata_path, index = 0)


                else:
                    raise ValueError("Pleas check model path !!")


            logging.info("There are {} events for each test.".format(len(validation_data)))
            logging.info("{} DNN models apply on {} is finished!!".format(model,traindata))
            logging.info("END===========================================END")
            logging.info("\n")


    logging.info("\n")
    logging.info("\n")


In [57]:
%%time
DNN_Model_A1 = {
#               "herwig_ang" : 0,
#               "pythia_def" : 0, 
#               "pythia_vin" : 0, 
              "pythia_dip" : 0, 
#               "sherpa_def" : 0,
            }

Performance_Frame = {
            "herwig_ang_train" : 0,
            "pythia_def_train" : 0,
            "pythia_vin_train" : 0,
            "pythia_dip_train" : 0,
        }  

        
# index=["Herwig Angular", "Pythia Default", "Pythia Vincia", "Pythia Dipole", "Average", "STD"]

for i, model in enumerate(DNN_Model_A1): 
    logging.info("DNN Model: {} ".format(model))
    

    
    for j, element in enumerate(Performance_Frame): 
        
        Performance = {
                    "AUC" : [0,0,0,0,0,0],
                    "AUC std" : [0,0,0,0,0,0],
                    "max_sig" : [0,0,0,0,0,0],
                    "max_sig std" : [0,0,0,0,0,0],
                    "r05" : [0,0,0,0,0,0],
                    "r05 std" : [0,0,0,0,0,0],
                    "time": [0,0,0,0,0,0],
                    "time std" : [0,0,0,0,0,0],
                    }

    
#         for k, (pt_min, pt_max) in  enumerate(zip([300,400,500,600,700,800],[400,500,600,700,800,900])):
        for k, (pt_min, pt_max) in  enumerate(zip([300],[500])):
            Performance_Table = pd.read_csv("./"+str(model)+"_KFold/DNN_"+str(model)+"_Performance_Table_to_"+str(element)+"_"+str(pt_min)+str(pt_max)+".csv")
#             Performance_Table = pd.read_csv("./"+str(model)+"_KFold/DNN_"+str(model)+"_Performance_Table_to_"+str(element)+"_"+str(pt_min)+str(pt_max)+".csv")
            
            logging.info(len(Performance_Table))

            Performance_Table.drop(Performance_Table[Performance_Table["AUC"] <= 0.7].index, inplace=True)
            Performance_Table.drop(Performance_Table[Performance_Table["AUC"] == 1].index, inplace=True)
            Performance["AUC"][k] = Performance_Table["AUC"].mean()
            Performance["AUC std"][k] = Performance_Table["AUC"].std()
            Performance["max_sig"][k] = Performance_Table["max_sig"].mean()
            Performance["max_sig std"][k] = Performance_Table["max_sig"].std()
            Performance["r05"][k] = Performance_Table["r05"].mean()
            Performance["r05 std"][k] = Performance_Table["r05"].std()
            Performance["time"][k] = Performance_Table["time"].mean()
            Performance["time std"][k] = Performance_Table["time"].std()



#         Performance_Frame[element] = pd.DataFrame(Performance,
#                      index=["pt300400","pt400500","pt500600","pt600700","pt700800","pt800900"]
#                     )
        
        Performance_Frame[element] = pd.DataFrame(Performance,
                     index=["pt3001100","null","null","null","null","null"]
                    )

INFO:root:DNN Model: pythia_dip 
INFO:root:100
INFO:root:100
INFO:root:100
INFO:root:100


CPU times: user 35.4 ms, sys: 2.48 ms, total: 37.9 ms
Wall time: 38.2 ms


In [58]:
Performance_Frame["herwig_ang_train"]

,AUC,AUC std,max_sig,max_sig std,r05,r05 std,time,time std
pt3001100,0.804079,0.009146,2.139877,0.591092,9.628607,1.061077,0.003058,0.001186
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [59]:
Performance_Frame["pythia_def_train"]

,AUC,AUC std,max_sig,max_sig std,r05,r05 std,time,time std
pt3001100,0.852991,0.017271,3.463959,0.935672,18.641138,2.959463,0.00312,0.001077
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [60]:
Performance_Frame["pythia_vin_train"]

,AUC,AUC std,max_sig,max_sig std,r05,r05 std,time,time std
pt3001100,0.863612,0.018464,5.542147,1.590907,25.419654,4.506597,0.002867,0.000512
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
Performance_Frame["pythia_dip_train"]

,AUC,AUC std,max_sig,max_sig std,r05,r05 std,time,time std
pt3001100,0.864795,0.007265,3.861803,0.997022,22.994514,3.312915,0.002854,0.000691
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
null,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
metric = "AUC"

print(np.mean([Performance_Frame["herwig_ang_train"][metric],Performance_Frame["pythia_def_train"][metric],Performance_Frame["pythia_vin_train"][metric],Performance_Frame["pythia_dip_train"][metric]], axis=0))
print(np.std([Performance_Frame["herwig_ang_train"][metric],Performance_Frame["pythia_def_train"][metric],Performance_Frame["pythia_vin_train"][metric],Performance_Frame["pythia_dip_train"][metric]], axis=0))



[0.84636908 0.         0.         0.         0.         0.        ]
[0.02484519 0.         0.         0.         0.         0.        ]


In [76]:
np.mean([0.854 ,0.860 ,0.862 ,0.865])

0.86025

In [75]:
np.std([0.854 ,0.860 ,0.862 ,0.865])

0.004023369234857774

In [97]:
%%time
DNN_Model_A1 = {
              "herwig_ang" : 0,
              "pythia_def" : 0, 
              "pythia_vin" : 0, 
              "pythia_dip" : 0, 
#               "sherpa_def" : 0,
            }

Performance_Frame = {
#             "herwig_ang_train" : 0,
#             "pythia_def_train" : 0,
#             "pythia_vin_train" : 0,
            "pythia_dip_train" : 0,
        }  

        
# index=["Herwig Angular", "Pythia Default", "Pythia Vincia", "Pythia Dipole", "Average", "STD"]


    

    
for j, element in enumerate(Performance_Frame): 
        
    for i, model in enumerate(DNN_Model_A1): 
        logging.info("DNN Model: {} ".format(model))
        
        Performance = {
                    "AUC" : [0,0,0,0,0,0],
                    "AUC std" : [0,0,0,0,0,0],
                    "max_sig" : [0,0,0,0,0,0],
                    "max_sig std" : [0,0,0,0,0,0],
                    "r05" : [0,0,0,0,0,0],
                    "r05 std" : [0,0,0,0,0,0],
                    "time": [0,0,0,0,0,0],
                    "time std" : [0,0,0,0,0,0],
                    }

    
#         for k, (pt_min, pt_max) in  enumerate(zip([300,400,500,600,700,800],[400,500,600,700,800,900])):
        for k, (pt_min, pt_max) in  enumerate(zip([300],[500])):
            Performance_Table = pd.read_csv("./"+str(model)+"_KFold/DNN_"+str(model)+"_Performance_Table_to_"+str(element)+"_"+str(pt_min)+str(pt_max)+".csv")
#             Performance_Table = pd.read_csv("./"+str(model)+"_KFold/DNN_"+str(model)+"_Performance_Table_to_"+str(element)+"_"+str(pt_min)+str(pt_max)+".csv")
            
            logging.info(len(Performance_Table))

            Performance_Table.drop(Performance_Table[Performance_Table["AUC"] <= 0.7].index, inplace=True)
            Performance_Table.drop(Performance_Table[Performance_Table["AUC"] == 1].index, inplace=True)
            Performance["AUC"][k] = Performance_Table["AUC"].mean()
            Performance["AUC std"][k] = Performance_Table["AUC"].std()
            Performance["max_sig"][k] = Performance_Table["max_sig"].mean()
            Performance["max_sig std"][k] = Performance_Table["max_sig"].std()
            Performance["r05"][k] = Performance_Table["r05"].mean()
            Performance["r05 std"][k] = Performance_Table["r05"].std()
            Performance["time"][k] = Performance_Table["time"].mean()
            Performance["time std"][k] = Performance_Table["time"].std()



#         Performance_Frame[element] = pd.DataFrame(Performance,
#                      index=["pt300400","pt400500","pt500600","pt600700","pt700800","pt800900"]
#                     )
        
        DNN_Model_A1[model] = pd.DataFrame(Performance,
                     index=["pt3001100","null","null","null","null","null"]
                    )

INFO:root:DNN Model: herwig_ang 
INFO:root:100
INFO:root:DNN Model: pythia_def 
INFO:root:100
INFO:root:DNN Model: pythia_vin 
INFO:root:100
INFO:root:DNN Model: pythia_dip 
INFO:root:100


CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 30.9 ms


In [98]:
metric = "AUC"

print(np.mean([DNN_Model_A1["herwig_ang"][metric],DNN_Model_A1["pythia_def"][metric],DNN_Model_A1["pythia_vin"][metric],DNN_Model_A1["pythia_dip"][metric]], axis=0))
print(np.std([DNN_Model_A1["herwig_ang"][metric],DNN_Model_A1["pythia_def"][metric],DNN_Model_A1["pythia_vin"][metric],DNN_Model_A1["pythia_dip"][metric]], axis=0))



[0.86004701 0.         0.         0.         0.         0.        ]
[0.00384274 0.         0.         0.         0.         0.        ]
